## **INSTALLATION** **OF REQUIRED PACKAGES**

In [ ]:
!pip install rank-bm25



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.2 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=4b33641c0b1bc7cadf8e0144327a067e481a0e5db0e74b66f3b9dcf3cfe89978
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 69.8 MB/s eta 0:00:00


### **IMPORTING** **PACKAGES**

In [ ]:
import json
import pandas as pd
from collections import defaultdict
from rank_bm25 import BM25Okapi
import math
import nltk
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import nltk
import re
import pickle
nltk.download('stopwords')
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.




### **LOADING VALIDATION DATA**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
inputs=pd.read_json('/content/drive/MyDrive/validate_questions.json')
df=pd.DataFrame()
categories=['women', 'religion', 'politics', 'style & beauty', 'entertainment', 'culture & arts', 'sports', 'science & technology', 'travel', 'business', 'crime','education', 'healthy living', 'parents', 'food & drink' ]

# **QUERY GENERATION**

### **GENERATION OF TOKENIZED INPUT ARTICLES (QUERIES)**


In [ ]:
pattern = r'article: (.*)'
df = inputs['input'].str.extract(pattern)
queries=(df.values).tolist()
tokens_queries=[]
for i in range(0,len(queries)):
  tokens_queries.append([word.lower() for word in queries[i][0].split()])

'\ntokenized_queries_path = "/content/drive/MyDrive/tokenized_queries.pkl"\n!touch {tokenized_queries_path}\nwith open(tokenized_queries_path, \'wb\') as f:\n  for i in range(0,len(queries)):\n    #print(queries[i])\n    #for word in queries[i]:\n    #word_token=re.split(\'\\W+\',queries[i][0])\n    query=queries[i][0]\n    word_token = query.split()\n    tokens = [stemmer.stem(token) for token in word_token if token not in stop_words and token != \'\']\n      #print(tokens)\n    pickle.dump(tokens, f)'

### **GENERATION OF CATEGORIES LIST**

In [ ]:
profiles= inputs['profile']
categories_list=[]
for i in profiles:
    category_each=[]
    for j in range(0, len(i)):
      for key, cat in i[j].items():
          if key != 'category':
            continue
          category_each.append(cat)
    categories_list.append(category_each)
          #print(key,cat)


In [ ]:
#print(categories_list[len(i)-1])

['entertainment', 'entertainment', 'entertainment', 'entertainment', 'entertainment', 'entertainment', 'entertainment', 'entertainment', 'entertainment', 'entertainment', 'entertainment', 'entertainment', 'entertainment', 'entertainment', 'entertainment', 'entertainment', 'entertainment', 'entertainment', 'entertainment', 'travel', 'style & beauty', 'entertainment', 'entertainment', 'sports', 'sports']


### **GENERATION OF TOKENIZED ARTICLES (DOCUMENTS)**

In [ ]:

tokenized_documents=[]
corpus=[]
p=[]
profiles= inputs['profile']
for i in profiles:
    c=[]
    p=[]
    for j in range(0, len(i)):
        #p.append([word.lower() for word in i[j]['text'].split()])
        # print(i[j])
        for key, word in i[j].items():
          if key != 'text':
            continue
          word_token=re.split('\W+', word)
          tokens = [stemmer.stem(token) for token in word_token if token not in stop_words and token != '']
          p.append(tokens)
        #p.append(i[j]['text'].split())
        c.append(i[j]['text'])
    tokenized_documents.append(p)
    corpus.append(c)



'file_path = "/content/drive/MyDrive/test_dumping.pkl"\n!touch {file_path}\nwith open(file_path, \'wb\') as f:\n  for i in profiles:\n    c=[]\n    p=[]\n    for j in range(0, len(i)):\n        #p.append([word.lower() for word in i[j][\'text\'].split()])\n        # print(i[j])\n        for key, word in i[j].items():\n          if key != \'text\':\n            continue\n          word_token=re.split(\'\\W+\', word)\n          tokens = [stemmer.stem(token) for token in word_token if token not in stop_words and token != \'\']\n          #p.append(tokens)\n          pickle.dump(tokens,f)\n        #p.append(i[j][\'text\'].split())\n        c.append(i[j][\'text\'])\n    tokenized_documents.append(p)\n    corpus.append(c)\n\n\nwith open(file_path, \'rb\') as f:\n  data = pickle.load(f)\n\n# data contains the python objects\nprint(data[0])\nprint(type(data))\n\nprint(tokenized_documents[0])'

In [ ]:
print(tokenized_documents[0])

[['the', 'elector', 'colleg', 'cast', 'vote', 'presid', 'elect', 'trump', 'decemb', 'sworn', 'offic', 'januari', 'he', 'presid', 'mani', 'american', 'troubl', 'accept', 'fact'], ['republican', 'presidenti', 'nomine', 'donald', 'trump', 'disgrac', 'parti', 'embarrass', 'nation', 'he', 'consist', 'demonstr', 'tempera', 'judgment', 'background', 'humil', 'lead', 'countri'], ['clinton', 'polar', 'figur', 'throughout', 'career', 'in', 'roosevelt', 'island', 'speech', 'portray', 'fighter', 'but', 'ultim', 'clinton', 'publicli', 'account', 'legitim', 'question', 'rais', 'around', 'candidaci', 'go', 'away'], ['the', 'u', 's', 'power', 'militari', 'histori', 'world', 'util', 'polit', 'tool', 'retribut', 'the', 'govern', 'leader', 'must', 'best', 'make', 'right', 'decis', 'truth', 'american', 'peopl', 'provid', 'necessari', 'support', 'need', 'fulfil', 'militari', 'mission', 'unfortun', 'alway', 'case'], ['her', 'unconvent', 'announc', 'video', 'indic', 'time', 'differ', 'but', '18', 'month', 'g


## **DUMPING TOKENIZED DOCUMENTS INTO FILE**

In [ ]:
documents_file_path = "/content/drive/MyDrive/validate_tokenized_documents_file.pkl"
!touch {documents_file_path}
with open(documents_file_path, 'wb') as f:
    pickle.dump(tokenized_documents, f)


In [ ]:
documents_file_path = "/content/drive/MyDrive/validate_tokenized_documents_file.pkl"
with open(documents_file_path, 'rb') as f:
  tokenized_documents_loaded = pickle.load(f)


In [ ]:
print(tokenized_documents_loaded[3])

[['the', 'dismiss', 'suit', 'concern', 'opinion', 'piec', 'suggest', 'trump', 'campaign', 'would', 'open', 'russian', 'interfer', '2020', 'elect'], ['evgeni', 'freidman', 'cohen', 'partner', 'taxi', 'busi', 'year'], ['the', 'teenag', 'allegedli', 'use', 'assault', 'rifl', 'fatal', 'shoot', 'four', 'peopl', 'parent', 'new', 'jersey', 'home'], ['the', 'walkout', 'threaten', 'profit', 'one', 'biggest', 'shop', 'day', 'year'], ['pre', 'donald', 'trump', 'brief', 'attack', 'trip', 'visit', 'victim', 'la', 'vega', 'shoot'], ['the', 'fda', 'issu', 'warn', 'letter', 'seiz', 'nearli', '100', 'websit', 'domain', 'name'], ['he', 'pulitz', 'prize', 'nation', 'book', 'award', 'nation', 'book', 'circl', 'critic', 'award', 'year']]


In [ ]:
import torch
print(torch.cuda.is_available())

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# **RETRIEVAL MODELS**

### **BM25** **RETRIEVAL MODEL**

In [ ]:
bm25_file_path = "/content/drive/MyDrive/validate_BM25_output_file.pkl"
!touch {bm25_file_path}

def BM25(tokenized_queries,tokenized_documents, corpus):
    length_queries=len(tokenized_queries)
    scores_list=[]
    top_k_docs=[]
    for i in range(0,length_queries):
        bm25 = BM25Okapi(tokenized_documents[i])
        v=tokenized_queries[i]
        scores_documents = bm25.get_scores(v)
        scores_list.extend(scores_documents)
        print(i)
        top_documents=bm25.get_top_n(queries[i],corpus[i], n=2)
        top_k_docs.append(top_documents)
    with open(bm25_file_path, 'wb') as f:
      pickle.dump(top_k_docs, f)
    return top_k_docs
    #print(top_k_docs[0])
r=BM25(tokens_queries,tokenized_documents_loaded, corpus)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
len(tokens_queries)

1052

In [ ]:
with open(bm25_file_path, 'rb') as f:
  bm25_output = pickle.load(f)


### **BERT - Bidirectional Encoder Representations from Transformers**

In [ ]:
def bert(query,document):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased').to(device)
    #tokenization-query
    tokenized_query = tokenizer.encode(query,  return_tensors='pt').to(device)
    output_query = model(tokenized_query).last_hidden_state.mean(dim=1)

    #tokenization-document
    output_documents=[]
    tokenized_documents = [tokenizer.encode(doc,  return_tensors='pt').to(device) for doc in document]
    for i in tokenized_documents:
        e=model(i).last_hidden_state.mean(dim=1)
        output_documents.append(e)

    #similarity calculation
    similarity=[]
    for i in output_documents:
        similarity.append(torch.nn.functional.cosine_similarity(output_query, i))

    #sorting documents
    sorted_documents = sorted(zip(document, similarity), key=lambda x: x[1], reverse=True)
    top_n_articles = [doc for doc in sorted_documents][:2]
    #print(top_n_articles)
    return top_n_articles

In [ ]:
N=len(tokenized_documents)

bert_file_path = "/content/drive/MyDrive/validate_bert_output_file.pkl"
!touch {bm25_file_path}
bert_articles=[]
with open(bert_file_path, 'wb') as f:
  for i in range(0,N):
    b=bert(queries[i],tokenized_documents[i])
    print(b)
    #bert_articles.append(b)
    pickle.dump(b, f)
    print(i)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

[(['republican', 'presidenti', 'nomine', 'donald', 'trump', 'disgrac', 'parti', 'embarrass', 'nation', 'he', 'consist', 'demonstr', 'tempera', 'judgment', 'background', 'humil', 'lead', 'countri'], tensor([0.4747], device='cuda:0', grad_fn=<SumBackward1>)), (['the', 'elector', 'colleg', 'cast', 'vote', 'presid', 'elect', 'trump', 'decemb', 'sworn', 'offic', 'januari', 'he', 'presid', 'mani', 'american', 'troubl', 'accept', 'fact'], tensor([0.4607], device='cuda:0', grad_fn=<SumBackward1>))]
0
[(['call', 'presidenti', 'impeach', 'cast', 'shadow', 'modern', 'day', 'presid', 'howev', 'choru', 'impeach', 'seem', 'louder', 'past', 'year'], tensor([0.5031], device='cuda:0', grad_fn=<SumBackward1>)), (['republican', 'presidenti', 'nomine', 'donald', 'trump', 'disgrac', 'parti', 'embarrass', 'nation', 'he', 'consist', 'demonstr', 'tempera', 'judgment', 'background', 'humil', 'lead', 'countri'], tensor([0.4747], device='cuda:0', grad_fn=<SumBackward1>))]
1
[(['call', 'presidenti', 'impeach', 'c

In [ ]:
bert_file_path = "/content/drive/MyDrive/validate_bert_output_file.pkl"
loaded_bert_articles=[]
with open(bert_file_path, 'rb') as f:
  for i in range(N):
        loaded_b = pickle.load(f)
        loaded_bert_articles.append(loaded_b)

print(len(loaded_bert_articles))



1052


## **PROMPT GENERATION**

### **BM25 PROMPT GENERATION FUNCTION**

In [ ]:
def prompt_function_bm25(queries, categories_list, relevant_articles_list, corpus):
  profile_len=len(corpus)
  w=[]
  for i in range(profile_len):
    query=queries[i]
    category=categories_list[i]
    a1=relevant_articles_list[i][0]
    a2=relevant_articles_list[i][1]
    prompt1=f"""The  articles is '{a1 , a2}''\n Article: {query}\nTASK: Given an article  and the top two relevant documents retrieved, Find the most relevant category from the options. Categories: [travel, style & beauty, food & drink, sports, business, science & technology, education, politics, religion, crime, parents, women, healthy living, entertainment, culture & arts] ?"""
    w.append(prompt1)
  return w


bm25_prompts = prompt_function_bm25(queries, categories_list, bm25_output, corpus)


### **BERT PROMPT GENERATION**

In [ ]:
def prompt_function_bm25(queries, categories_list, relevant_articles_list, corpus):
  profile_len=len(corpus)
  w=[]
  for i in range(profile_len):
    query=queries[i]
    category=categories_list[i]
    a1=relevant_articles_list[i][0][0]
    #print(a1)
    a2=relevant_articles_list[i][1][0]
    #print(a2)
    prompt1=f"""The  article is '{' '.join(a1), ' '.join(a2)}'\nArticle: '{query[0]}' \n TASK: Given an article  and the top two relevant documents retrieved, find the most relevant category from the given options. Categories: [travel, style & beauty, food & drink, sports, business, science & technology, education, politics, religion, crime, parents, women, healthy living, entertainment, culture & arts] ?"""
    w.append(prompt1)
    #prompts.append(prompt)
  return w


bert_prompts = prompt_function_bm25(queries, categories_list, loaded_bert_articles, corpus)

In [ ]:
bert_prompts[1]

"The  article is '('call presidenti impeach cast shadow modern day presid howev choru impeach seem louder past year', 'republican presidenti nomine donald trump disgrac parti embarrass nation he consist demonstr tempera judgment background humil lead countri')'\nArticle: 'With less than 10 weeks to go before the midterm Congressional elections Americans in general are frustrated with Washington. National polls show that about three-quarters of all Americans disapprove of the way Congress is doing its job. Because so much is at stake, this coming election day is not a time for eligible voters to stay home.' \n TASK: Given an article  and the top two relevant documents retrieved, find the most relevant category from the given options. Categories: [travel, style & beauty, food & drink, sports, business, science & technology, education, politics, religion, crime, parents, women, healthy living, entertainment, culture & arts] ?"

# **LARGE LANGUAGE MODEL**

## **FLAN-T5 BASE**

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")

final_bm25_outputs=[]
for i in bm25_prompts:
  input_ids = tokenizer(i, return_tensors="pt").input_ids
  outputs = model.generate(input_ids)
  final_bm25_outputs.append(tokenizer.decode(outputs[0]))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")

final_outputs=[]
for i in bert_prompts:
  input_ids = tokenizer(i, return_tensors="pt").input_ids
  outputs = model.generate(input_ids)
  final_outputs.append(tokenizer.decode(outputs[0]))


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
len(final_outputs)


2104

# **EVALUATION OF VALIDATION OUTPUTS VS OBTAINED OUTPUTS**

### **BM25**

In [ ]:
#final_bm25_outputs=[]
d=inputs['id'].tolist()
final_outputs_bm25=[]
prompts_df = pd.DataFrame(bm25_prompts, columns=['prompts'])
#bm25_prompts
for i,data in inputs.iterrows():
  y=0
  input_text = prompts_df['prompts'][i]
  #print(input_text)
  input_ids = tokenizer(input_text, return_tensors="pt").input_ids
  outputs = model.generate(input_ids)
  result = tokenizer.decode(outputs[0], skip_special_tokens=True)
  values= {}
  values["id"] = str(data["id"])
  print(values["id"])
  values["output"] = result
  final_outputs_bm25.append(values)
  y+=1

bm25_output_json = { "task" : "LaMP_2", "golds" : final_outputs_bm25}

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


110
111
112
113
114
115
116
117
118
119
1110
1111
1112
1113
1114
1115
1116
1117
1118
1119
1120
1121
1122
1123
1124
1125
1126
1127
1128
1129
1130
1131
1132
1133
1134
1135
1136
1137
1138
1139
1140
1141
1142
1143
1144
1145
1146
1147
1148
1149
1150
1151
1152
1153
1154
1155
1156
1157
1158
1159
1160
1161
1162
1163
1164
1165
1166
1167
1168
1169
1170
1171
1172
1173
1174
1175
1176
1177
1178
1179
1180
1181
1182
1183
1184
1185
1186
1187
1188
1189
1190
1191
1192
1193
1194
1195
1196
1197
1198
1199
11100
11101
11102
11103
11104
11105
11106
11107
11108
11109
11110
11111
11112
11113
11114
11115
11116
11117
11118
11119
11120
11121
11122
11123
11124
11125
11126
11127
11128
11129
11130
11131
11132
11133
11134
11135
11136
11137
11138
11139
11140
11141
11142
11143
11144
11145
11146
11147
11148
11149
11150
11151
11152
11153
11154
11155
11156
11157
11158
11159
11160
11161
11162
11163
11164
11165
11166
11167
11168
11169
11170
11171
11172
11173
11174
11175
11176
11177
11178
11179
11180
11181
11182
11183
11184


In [ ]:

json_data = json.dumps(final_outputs_bm25, indent=2)
location= "/content/drive/MyDrive/bm25_output.json"
with open(location, "w") as jf:
    jf.write(json_data)

### **BERT**

In [ ]:
#final_outputs=[]
final_outputs_bert=[]
print(bert_prompts[8])
promptsb_df = pd.DataFrame(bert_prompts, columns=['prompts'])

for i,data in inputs.iterrows():
  y=0
  input_text = promptsb_df['prompts'][i]
  input_ids = tokenizer(input_text, return_tensors="pt").input_ids
  outputs = model.generate(input_ids)
  result = tokenizer.decode(outputs[0], skip_special_tokens=True)
  values= {}
  values["id"] = str(data["id"])
  values["output"] = result
  final_outputs_bert.append(values)
  y=y+1

bert_output_json = { "task" : "LaMP_2", "golds" : final_outputs_bert}







The  article is '('she afraid speak believ', 'they up down video pretti ador')'
Article: 'Yet another clue that lots of Harry music is definitely coming our way! 🙌' 
 TASK: Given an article  and the top two relevant documents retrieved, find the most relevant category from the given options. Categories: [travel, style & beauty, food & drink, sports, business, science & technology, education, politics, religion, crime, parents, women, healthy living, entertainment, culture & arts] ?


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


KeyboardInterrupt: ignored

In [ ]:

json_data = json.dumps(bert_output_json, indent=2)
location= "/content/drive/MyDrive/bert_output.json"
with open(location, "w") as jf:
    jf.write(json_data)

In [ ]:
len(final_outputs_bert)

1052